In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [101]:
drop_columns = ['pitch_type', 'spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated', 'break_length_deprecated', 'tfs_deprecated', 'tfs_zulu_deprecated', 'umpire', "game_date", "game_year", "des", "player_name"]
df_raw = pd.read_csv("/Users/ianchang/Library/Mobile Documents/com~apple~CloudDocs/1. Project/Baseball/baseball/raw-data-ignore/statcast_pitch_swing_data_20240402_20240630.csv", low_memory=False).drop(columns=drop_columns)
df_no_bunt = df_raw[~df_raw['description'].str.contains('bunt', case=False, na=False)].copy()

num_cols = ['release_speed', 'release_pos_x', 'release_pos_z', 'plate_x', 'plate_z', 'hit_distance_sc', 
            'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate']
df_no_bunt[num_cols] = df_no_bunt[num_cols].apply(pd.to_numeric, errors='coerce')

df_no_bunt = df_no_bunt[~df_no_bunt["description"].str.contains("pitchout", case=False, na=False)]
df_no_bunt = df_no_bunt[~df_no_bunt["description"].str.contains("hit_by_pitch", case=False, na=False)]
df_no_bunt = df_no_bunt[~df_no_bunt["events"].str.contains("sac_bunt", case=False, na=False)]
# Filter out rows where 'swing_length' is NaN and 'description' is "hit_into_play" cuz it kindof makes no sense
df_no_bunt = df_no_bunt[~((pd.isna(df_no_bunt['swing_length'])) & (df_no_bunt['description'] == "hit_into_play"))]

hits = ["single", "home_run", "double", "triple"]

for index, row in df_no_bunt.iterrows():
    if row["events"] in hits:
        replacement = row["events"]
        df_no_bunt.at[index, "description"] = replacement 
    if row["description"] == "hit_into_play":
        replacement = "hit_caught_out"
        df_no_bunt.at[index, "description"] = replacement

df_no_bunt = df_no_bunt.drop(columns=["events"])
'''
'''

'\n'

In [102]:
df_no_bunt.description.unique()

array(['foul', 'called_strike', 'ball', 'foul_tip', 'swinging_strike',
       'blocked_ball', 'swinging_strike_blocked', 'hit_caught_out'],
      dtype=object)

In [103]:
homeruns = df_no_bunt[df_no_bunt.description == "home_run"]
singles = df_no_bunt[df_no_bunt.description == "single"]
double = df_no_bunt[df_no_bunt.description == "double"]
triple = df_no_bunt[df_no_bunt.description == "triple"]

In [104]:
df_no_bunt[(df_no_bunt.description == "hit_into_play") & (pd.isna(df_no_bunt.swing_length))]

,release_speed,release_pos_x,release_pos_z,batter,pitcher,description,zone,game_type,stand,p_throws,...,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length


In [ ]:
df_no_bunt.to_csv("/Users/ianchang/Library/Mobile Documents/com~apple~CloudDocs/1. Project/Baseball/baseball/Data Cleaning/cleaned_set_r1.csv", index=False)